# Profitable App Profiles for the App Store and Google Play Markets

The project is devoted to free to download iOS and Android apps analysis. We assume, we're working for a company that builds Android and iOS apps and our aim is to find mobile app profiles which are profitable for App Store and Google Play markets.

We'll analyse free Android and iOS apps to find which apps categories are the most popular amoung useres and on this basis, we'll propose a type of app which may has the biggest chance to gain more popularity than another app profiles.

Apple Store data set is available [here](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)

Google Play data set is available [here](https://www.kaggle.com/lava18/google-play-store-apps)



Let's explore our data sets. First, we will open and read both files: *AppleStore.csv* and *googleplaystore.csv*

In [1]:
from csv import reader

#Apple Store dataset
oppend_file = open('AppleStore.csv', encoding="utf8")
read_file = reader(oppend_file)
ios_apps_data = list(read_file)

#Google Play dataset
oppend_file = open('googleplaystore.csv', encoding="utf8")
read_file = reader(oppend_file)
android_apps_data = list(read_file)


def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [2]:
explore_data(android_apps_data, 0, 2, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13


In [3]:
explore_data(ios_apps_data, 0, 2, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows: 7198
Number of columns: 16


Description of each column in Apple Store data set: [Link](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

In [4]:
# Delete one row [10473] index, one column is missing there
del android_apps_data[10473]

In [5]:
#Checking duplicates for Apple Dataset

ios_unique_apps = [] 
ios_duplicate_apps = [] 

for app in ios_apps_data: 
    app_id = app[0] 

    if app_id in ios_unique_apps:
        ios_duplicate_apps.append(app_id)
    else:
        ios_unique_apps.append(app_id)

In [6]:
print("The number of duplicates in Apple Store Dataset: ", len(ios_duplicate_apps))

The number of duplicates in Apple Store Dataset:  0


In [7]:
#Checking duplicates for Google Play Dataset

android_unique_apps = [] 
android_duplicate_apps = [] 

for app in android_apps_data: 
    app_name = app[0] 

    if app_name in android_unique_apps:
        android_duplicate_apps.append(app_name)
    else:
        android_unique_apps.append(app_name)

In [8]:
print("The number of duplicates in Google Play Dataset: ", len(android_duplicate_apps))
print("3 examples: \n")

explore_data(android_duplicate_apps, 0, 3)

The number of duplicates in Google Play Dataset:  1181
3 examples: 

Quick PDF Scanner + OCR FREE


Box


Google My Business




We can't remove dupliactes randomly. We have to decide, which duplacte we will remove. 
Our duplicates contain diffrent number of reviews, so we will choose row-app with the most user reviews. 
The others with less numbr of reviews we will delete from dataset.

1. We created an empty **dictionary** to find one app with the highest number of reviews  among our duplacates.
2. We loop through the Google Play data set.
3. We adding name of app as a key and number of reviews as a value to our *reviews_max* **dictonary**.
4. If we find the duplicate and the number of reviews is higher than our already exists item in *reviews_max* **dictonary**, we uppdate this value. 


In [9]:
reviews_max = {}

for app in android_apps_data[1:]:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] =  n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

In [10]:
len(reviews_max)

9659

Next, we will clean our data set. 

1. We created two empty lists: *android_clean* and *already_added*.
2. We loop through the android data set.
3. We're checking if current number of reviews is equal to our value in *reviews_max* **dictionary** and we have to check also if app row hasn't been added before. Some duplicates has the same number of review. The *already_added* **list** contains the unqiue apps name

In [11]:
android_clean = []
already_added = []

for app in android_apps_data[1:]:
    name = app[0]
    n_reviews = float(app[3])
    
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)

In [12]:
len(android_clean)

#After cleaning process, our Google Play data set contains 9659 rows

9659

In [13]:
len(ios_apps_data[1:])

#Just for record, App Store has 7197 rows

7197

In [14]:
def check_if_eng(name):
    for letter in name:
        if ord(letter) > 127:
            return False
    return True
    

In [15]:
print(check_if_eng('Docs To Go™ Free Office Suite'))
print(check_if_eng('Instachat 😜'))
print(check_if_eng('爱奇艺PPS -《欢乐颂2》电视剧热播'))


False
False
False


We may see that our created **function** - *check_if_eng(name)* is not perfect. Some symbols like (™) is outside our range for english letters.

We will uppdate our *check_if_eng(name)* **function**

In [16]:
def check_if_eng(name):
    none_eng_letters = 0
    
    for letter in name:
        if ord(letter) > 127:
            none_eng_letters += 1
    if none_eng_letters > 3:
        return False
    else:
        return True

We assume that 3 letter from out of our range are acceptable to treats app name as english.

In [17]:
#Deleting no-english apps in Google Play data set
eng_android_apps_data = []

for row in android_clean:
    name = row[0]
    eng_name = check_if_eng(name)
    if eng_name:
        eng_android_apps_data.append(row)
        

In [18]:
len(eng_android_apps_data)
#We have 9614 english apps in Google Play data set now 

9614

In [19]:
#Deleting no-english apps in App Store data set
eng_ios_apps_data = []

for row in ios_apps_data[1:]:
    name = row[1]
    eng_name = check_if_eng(name)
    if eng_name:
        eng_ios_apps_data.append(row)
        

In [20]:
len(eng_ios_apps_data)
#We have 6183 english apps in App Store data set now 

6183

In [21]:
android_free_apps = []

for row in eng_android_apps_data:
    price = row[7]
    if (price == '0') or (price == '0.0'):
        android_free_apps.append(row)

In [22]:
len(android_free_apps)

8864

In [23]:
ios_free_apps = []

for row in eng_ios_apps_data:
    price = row[4]
    if (price == '0') or (price == '0.0'):
        ios_free_apps.append(row)

In [24]:
len(ios_free_apps)

3222

After cleaning process we have:
* 8873 Android free apps,
* 3300 iOS free apps.


Next, we will find the most popular app category in either Google Play or App Store.
It allows to build an app, which will has the greatest chance to get to the highest number of users in both markets. 
App will create on Android first, then if will be a good users' response, our develop department put this app also in App Store after six months.


In [25]:
def freq_table(dataset, index):
    freqDict = {}
    for row in dataset:
        key = row[index]
        if key in freqDict:
            freqDict[key] += 1
        else:
            freqDict[key] = 1
            
    values_sum = sum(freqDict.values())
        
    for key, value in freqDict.items():
        perc = value / values_sum * 100
        freqDict[key] = perc
        
    return freqDict

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

*Freq_table()* **function** is showing frequency of specific column in percentage.
*Display_table()* **function** prints all items from *Freq_table()* **function**, sorted by values.
In this case we're checking which category has the most apps.

Let's see which categories have the most apps in Google Play.

In [26]:
display_table(android_free_apps, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

Judging by the result above, the most free and english apps available on Android are in categories:
 * FAMILY 19%
 * GAME 10%
 * TOOLS 8,5%
 * BUSINESS 4,5%
 
 

In [27]:
display_table(android_free_apps, -4)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

We may noticed that the **Catogory** and **Genres** columns is not obvious. **Genres** column)  has much categories.
If we look again on both columns, we can spot, that *FAMILY* category (19% share in **Catogory** column) and *Entertainment* genre (6% share in **Genres** column), which could mean that the most apps are games dedicated to entire family, including children.

In [28]:
display_table(ios_free_apps, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Analyzig free and english apps on iOS, we see that **Games** category has the most apps available in App Store. Next is Entertainment and this may contains appps connected with games also.

We can make it clear that the largest amonut free and english iOS apps provides users fun and entertainment.

At the moment, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps.

But does these information indicate on the most popular categories amoung useres also? Well, we have to examine which categories have the most number of users.

# Most Popular Apps by Genre on App Store

Let's see which categories have the most useres average. 



In [29]:
ios_freq_genere = freq_table(ios_free_apps, -5)

for genere in ios_freq_genere:
    total = 0
    len_genere = 0
    
    
    for app in ios_free_apps:
        genre_app = app[-5]
        
        if genre_app == genere:
            total_user_rating = float(app[5])
            total += total_user_rating
            len_genere += 1
            
    avg_user = round(total / len_genere)
    print(genere, ' : ', avg_user)

Social Networking  :  71548
Photo & Video  :  28442
Games  :  22789
Music  :  57327
Reference  :  74942
Health & Fitness  :  23298
Weather  :  52280
Utilities  :  18684
Travel  :  28244
Shopping  :  26920
News  :  21248
Navigation  :  86090
Lifestyle  :  16486
Entertainment  :  14030
Food & Drink  :  33334
Sports  :  23009
Book  :  39758
Finance  :  31468
Education  :  7004
Productivity  :  21028
Business  :  7491
Catalogs  :  4004
Medical  :  612


We can see that on avarage, the most popular categories in App Store are:
 * Navigation (86090),
 * Reference  (74942), 
 * Social Networking (71548),
 * Music (57327),
 * Weather (52280).
 
We know now that amount of apps in each category, dosen't alwyas points on the most popular apps amoung useres.

In [30]:
for app in ios_free_apps:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5]) # print name and number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


Ok, **Navigation** category is dominated by *Waze* and *Google Maps*. The rest apps has a little number of ratings.

We can supposed that, similar situation is in Social Networking category. There are few giants, like: *Facebook, Instagram, Twitter, YouTube.* etc.

**Weather** category has many users, but the problem is that people spends too little time during checking the actual wetaher.



In [31]:
for app in ios_free_apps:
    if app[-5] == 'Music':
        print(app[1], ':', app[5]) # print name and number of ratings

Pandora - Music & Radio : 1126879
Spotify Music : 878563
Shazam - Discover music, artists, videos & lyrics : 402925
iHeartRadio – Free Music & Radio Stations : 293228
SoundCloud - Music & Audio : 135744
Magic Piano by Smule : 131695
Smule Sing! : 119316
TuneIn Radio - MLB NBA Audiobooks Podcasts Music : 110420
Amazon Music : 106235
SoundHound Song Search & Music Player : 82602
Sonos Controller : 48905
Bandsintown Concerts : 30845
Karaoke - Sing Karaoke, Unlimited Songs! : 28606
My Mixtapez Music : 26286
Sing Karaoke Songs Unlimited with StarMaker : 26227
Ringtones for iPhone & Ringtone Maker : 25403
Musi - Unlimited Music For YouTube : 25193
AutoRap by Smule : 18202
Spinrilla - Mixtapes For Free : 15053
Napster - Top Music & Radio : 14268
edjing Mix:DJ turntable to remix and scratch music : 13580
Free Music - MP3 Streamer & Playlist Manager Pro : 13443
Free Piano app by Yokee : 13016
Google Play Music : 10118
Certified Mixtapes - Hip Hop Albums & Mixtapes : 9975
TIDAL : 7398
YouTube Mu

**Musis** *category* is also dominated by few big players like Pandora, Spotify, and Shazam. They make haeavliy inlfuence on average number.
But we can noticed some others Music apps which have many useres.

In [32]:
for app in ios_free_apps:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5]) # print name and number of ratings

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


Here, in **Reference** category, the main app is Bible and makes big influence on average number. 

# Most Popular Apps by Genre on Google Play

Let's check our data in Google Play apps. Previous, we had accurate date about number of votes. Now, we have information about the number of installs. At face value, number of downloads seems to be more accurate than number of votes, but in this case we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):



In [33]:
display_table(android_free_apps, 5) #the Installs columns

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


Above, we noticed that our data is not precise. For instance, if we take closer look on apps with 1,000,000+ installs, we don't now if is 1,000,000 installs, 2,000,000 or 4,500,000. The next range is 5,000,000 installs.

However, we don't need accurate data also. We have to find only category which is the most popular amoung users, not specific apps.

In this case, we assume that 1,000,000+ installs is the number 1,000,000, 50,000+ is 50,000 etc.

We have to convert our data in **Installs** *columns* from strings to floats and remove ',' and '+' characters, becasue we will get an error if we don't do it.

In [34]:
android_freq_genere = freq_table(android_free_apps, 1)

for category in android_freq_genere:
    total = 0
    len_category = 0
    
    for app in android_free_apps:
        category_app = app[1]
        if category_app == category:
            installs = app[5]
            installs = float(installs.replace('+','').replace(',', ''))            
            total += installs
            len_category += 1
            
    avg_user = round(total / len_genere)
    if avg_user > 100000000:
        print(category, ' : ', avg_user)

BOOKS_AND_REFERENCE  :  277647377
BUSINESS  :  116150348
COMMUNICATION  :  1839484367
ENTERTAINMENT  :  164910000
HEALTH_AND_FITNESS  :  190591400
GAME  :  2239478242
FAMILY  :  1032315948
SOCIAL  :  914643650
SHOPPING  :  233389764
PHOTOGRAPHY  :  776044802
SPORTS  :  182538447
TRAVEL_AND_LOCAL  :  482450681
TOOLS  :  1350173912
PERSONALIZATION  :  254872648
PRODUCTIVITY  :  965271552
VIDEO_PLAYERS  :  655288620
NEWS_AND_MAGAZINES  :  394699377


We see that **Game** has 2,239,478,242 installs.

In [35]:
for app in android_free_apps:
    if app[1] == 'GAME' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Sonic Dash : 100,000,000+
PAC-MAN : 100,000,000+
Roll the Ball® - slide puzzle : 100,000,000+
Piano Tiles 2™ : 100,000,000+
Pokémon GO : 100,000,000+
Extreme Car Driving Simulator : 100,000,000+
Trivia Crack : 100,000,000+
Angry Birds 2 : 100,000,000+
Candy Crush Saga : 500,000,000+
8 Ball Pool : 100,000,000+
Subway Surfers : 1,000,000,000+
Candy Crush Soda Saga : 100,000,000+
Clash Royale : 100,000,000+
Clash of Clans : 100,000,000+
Plants vs. Zombies FREE : 100,000,000+
Pou : 500,000,000+
Flow Free : 100,000,000+
My Talking Angela : 100,000,000+
slither.io : 100,000,000+
Cooking Fever : 100,000,000+
Yes day : 100,000,000+
Score! Hero : 100,000,000+
Dream League Soccer 2018 : 100,000,000+
My Talking Tom : 500,000,000+
Sniper 3D Gun Shooter: Free Shooting Games - FPS : 100,000,000+
Zombie Tsunami : 100,000,000+
Helix Jump : 100,000,000+
Crossy Road : 100,000,000+
Temple Run 2 : 500,000,000+
Talking Tom Gold Run : 100,000,000+
Agar.io : 100,000,000+
Bus Rush: Subway Edition : 100,000,00

**Game** *category* is very crowded by a lot of games. They are very popular amoung useres, but couldbe difficult to create game app which will attract attention fastly.

Let's look at **Family** *category*
 

In [36]:
for app in android_free_apps:
    if app[1] == 'FAMILY' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Minion Rush: Despicable Me Official Game : 100,000,000+
ROBLOX : 100,000,000+
Hay Day : 100,000,000+
Talking Tom Cat 2 : 100,000,000+
Talking Ginger : 100,000,000+
Where's My Water? Free : 100,000,000+
Talking Tom Cat : 100,000,000+
Tom Loves Angela : 100,000,000+
Talking Tom & Ben News : 100,000,000+
Google Play Games : 1,000,000,000+
Bitmoji – Your Personal Emoji : 100,000,000+
Bubble Witch 2 Saga : 100,000,000+
Duolingo: Learn Languages Free : 100,000,000+
Farm Heroes Saga : 100,000,000+


There is no such apps with 1,000,000,000+ installs and more in this category. As we can see, there are also many games.  

In [37]:

for app in android_free_apps:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

As we can spot, there are many popular apps there and dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries. 

Remember that similar results we got in App Store apps, so it's seems that creat an app in **Refernce** *category* is good proposal to our develop team. We can creat an app which will read books. Additionally, to make stand out our app, we will add audiobook option. Speech synthesizer will read the toxt from the book. It will be a great benfit to user, because he will read the books faster. 



## Conclusion

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that creating an app in Reference category is good idea, but we have to remember, ordinary e-book reader 
will not attract many users. Adding audiobook function should increase popularity. Moreover, pepole during reading and listing their favourite books will spend a lot of time, so it is a huge profit for us.

